In [ ]:
"""
author: Vetivert? 💐 
created: 13/03/2025 @ 12:46:33
"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from prefix_path import full_data_path, redundant_cols

In [ ]:
# Load dataset
df = pd.read_csv(full_data_path, sep=";", header=3) #separator is ;

# Drop redundant cols + rows
df.drop([224, 225, 226]) #indexes of 2019 data


  ID BN  STT           Full name Age Diagnosis Gende Blood group  \
0      2   1       TRẦN DUY CHÀO  46         2     1         NaN   
1      3   2    NGUYỄN VIẾT XUÂN  35         1     1         NaN   
2      4   3  BÙI THỊ THANH DUNG  25         1     0         NaN   
3      5   4           CỨ A TỈNH  20         2     1           B   
4      6   5    NGUYỄN VĂN QUÂN   31         1     1           O   

               Ward             District Hospitalization day  ...  \
0               NaN  Thanh Trì - Hà Nội             30/10/17  ...   
1           Phúc La    Hà Đông - Hà Nội             30/10/17  ...   
2               NaN    Hà Đông - Hà Nội             30/10/17  ...   
3  Học viện an ninh  Thanh Xuân - Hà Nội            31/10/17  ...   
4         Lương Đô    Thanh Ba - Phú Thọ            22/10/17  ...   

  Unnamed: 511 Unnamed: 512 Unnamed: 513 Unnamed: 514 Unnamed: 515  \
0          NaN          NaN          NaN          NaN          NaN   
1          NaN          NaN         

In [ ]:
df.drop(redundant_cols, axis=1)

,Full name,Age,Diagnosis,Gende,Blood group,Ward,District,Hospitalization day,Discharge day,Patient No,...,Unnamed: 511,Unnamed: 512,Unnamed: 513,Unnamed: 514,Unnamed: 515,Unnamed: 516,Unnamed: 517,Unnamed: 518,Unnamed: 519,Unnamed: 520
0,TRẦN DUY CHÀO,46,2,1,NaN,NaN,Thanh Trì - Hà Nội,30/10/17,3/11/17,4280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NGUYỄN VIẾT XUÂN,35,1,1,NaN,Phúc La,Hà Đông - Hà Nội,30/10/17,11/7/17,Q542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BÙI THỊ THANH DUNG,25,1,0,NaN,NaN,Hà Đông - Hà Nội,30/10/17,7/11/17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CỨ A TỈNH,20,2,1,B,Học viện an ninh,Thanh Xuân - Hà Nội,31/10/17,11/7/17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NGUYỄN VĂN QUÂN,31,1,1,O,Lương Đô,Thanh Ba - Phú Thọ,22/10/17,26/10/17,4055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Phùng Thị Lũy,37,2,0,O,NGUYỄN TRÃI,"THƯỜNG TÍN, HÀ NỘI",15/12/19,20/12/19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Triệu Thị Hải,37,2,0,NaN,CẨM SƠN,"CẨM THỦY, THANH HÓA",16/12/19,20/12/19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Bùi Viết Tuấn,19,1,1,O,THÁI SƠN,"AN LÃO, HẢI PHÒNG",28/8/19,09/04/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,Đỗ Mạnh Thắng,44,2,1,NaN,"VĂN QUÁN, HÀ ĐÔNG",HÀ NỘI,17/10/19,22/10/19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
